In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
vep_cols = """\
Allele|Consequence|IMPACT|SYMBOL|Gene|Feature_type|Feature|BIOTYPE|EXON\
|INTRON|HGVSc|HGVSp|cDNA_position|CDS_position|Protein_position|Amino_acids\
|Codons|Existing_variation|DISTANCE|STRAND|FLAGS|SYMBOL_SOURCE|HGNC_ID|CANONICAL\
|CCDS|HGVS_OFFSET\
"""
vep_cols = vep_cols.split("|")
vep_cols = [term.strip().capitalize() for term in vep_cols]

In [3]:
vep_df = pd.read_table('./cftr.grch37.vep.vcf', header=0, skiprows=6, usecols=range(10))
info_df = vep_df["INFO"].str.replace("ANN=", "").str.split(",").apply(pd.Series, 1).stack()
info_df = info_df.str.split("|").apply(pd.Series, 1)
info_df.index = info_df.index.droplevel(-1)
info_df.columns = vep_cols
vep_df = vep_df.join(info_df)
vep_df = vep_df[['#CHROM', 'POS', 'REF', 'ALT', 'Consequence', 'Impact', 
                     'Symbol', 'Gene', 'Feature_type', 'Feature', 'Biotype', 
                     'Hgvsc','Hgvsp','Hgnc_id']]
del info_df
vep_df.head()

,#CHROM,POS,REF,ALT,Consequence,Impact,Symbol,Gene,Feature_type,Feature,Biotype,Hgvsc,Hgvsp,Hgnc_id
0,7,117105737,C,A,upstream_gene_variant,MODIFIER,CFTR,ENSG00000001626,Transcript,ENST00000546407,processed_transcript,,,1884
1,7,117105737,C,G,upstream_gene_variant,MODIFIER,CFTR,ENSG00000001626,Transcript,ENST00000546407,processed_transcript,,,1884
2,7,117105737,C,T,upstream_gene_variant,MODIFIER,CFTR,ENSG00000001626,Transcript,ENST00000546407,processed_transcript,,,1884
3,7,117105737,C,CA,upstream_gene_variant,MODIFIER,CFTR,ENSG00000001626,Transcript,ENST00000546407,processed_transcript,,,1884
4,7,117105737,C,CG,upstream_gene_variant,MODIFIER,CFTR,ENSG00000001626,Transcript,ENST00000546407,processed_transcript,,,1884


In [4]:
snpeff_cols = """\
Allele|Annotation|Putative_impact|Gene_name|Gene_ID|Feature_type|Feature_ID|Transcript_Biotype|Rank|HGVSc|HGVSp\
|cDNA_position|CDS_position|Protein_position|Distance|Errors"""
snpeff_cols = snpeff_cols.split("|")
snpeff_cols = [term.strip().capitalize() for term in snpeff_cols]

In [5]:
snpeff_df = pd.read_table('./cftr.grch37.snpeff.vcf', header=0, skiprows=9, usecols=range(10))
info_df = snpeff_df["INFO"].str.split(";").apply(pd.Series, 1)[0] #snpeff includes two other INFO fields that we don't need
info_df = info_df.str.replace("ANN=", "").str.split(",").apply(pd.Series, 1).stack()
info_df = info_df.str.split("|").apply(pd.Series, 1)
info_df.index = info_df.index.droplevel(-1)
info_df.columns = snpeff_cols
snpeff_df = snpeff_df.join(info_df)
snpeff_df = snpeff_df[['#CHROM', 'POS', 'REF', 'ALT', 'Annotation', 'Putative_impact','Gene_id', 'Feature_type', 'Feature_id', 'Transcript_biotype', 
                     'Hgvsc','Hgvsp']]
del info_df
snpeff_df.head()

,#CHROM,POS,REF,ALT,Annotation,Putative_impact,Gene_id,Feature_type,Feature_id,Transcript_biotype,Hgvsc,Hgvsp
0,7,117105737,C,A,upstream_gene_variant,MODIFIER,ENSG00000001626,transcript,ENST00000546407,processed_transcript,n.-101C>A,
0,7,117105737,C,A,intergenic_region,MODIFIER,ENSG00000214684-ENSG00000001626,intergenic_region,ENSG00000214684-ENSG00000001626,,n.117105737C>A,
1,7,117105737,C,G,upstream_gene_variant,MODIFIER,ENSG00000001626,transcript,ENST00000546407,processed_transcript,n.-101C>G,
1,7,117105737,C,G,intergenic_region,MODIFIER,ENSG00000214684-ENSG00000001626,intergenic_region,ENSG00000214684-ENSG00000001626,,n.117105737C>G,
2,7,117105737,C,T,upstream_gene_variant,MODIFIER,ENSG00000001626,transcript,ENST00000546407,processed_transcript,n.-101C>T,


In [ ]:
#From http://snpeff.sourceforge.net/VCFannotationformat_v1.0.pdf with additions:
#VEP: non_coding_transcript_exon_variant, non_coding_transcript_variant, protein altering variant, 
#incomplete_terminal_codon_variant, NMD_transcript_variant
#Snpeff: conservative_inframe_deletion, conservative_inframe_insertion, structural_interaction_variant, 5_prime_UTR_truncation

ranked_terms = ["chromosome_number_variation","exon_loss_variant","frameshift_variant","stop_gained","stop_lost",
                "start_lost","splice_acceptor_variant","splice_donor_variant","rare_amino_acid_variant","missense_variant",
                "inframe_insertion","conservative_inframe_insertion", "disruptive_inframe_insertion","inframe_deletion","conservative_inframe_deletion", "disruptive_inframe_deletion",
                "5_prime_UTR_truncation+exon_loss_variant","5_prime_UTR_truncation","3_prime_UTR_truncation+exon_loss","splice_branch_variant",
                "splice_region_variant","splice_branch_variant","stop_retained_variant","initiator_codon_variant",
                "synonymous_variant","initiator_codon_variant+non_canonical_start_codon","stop_retained_variant",
                "coding_sequence_variant","5_prime_UTR_variant","3_prime_UTR_variant",
                "5_prime_UTR_premature_start_codon_gain_variant","structural_interaction_variant", "protein_altering_variant","upstream_gene_variant","downstream_gene_variant",
                "TF_binding_site_variant","regulatory_region_variant","miRNA","custom","sequence_feature",
                "conserved_intron_variant","intron_variant","intragenic_variant","conserved_intergenic_variant",
                "intergenic_region","coding_sequence_variant","non_coding_exon_variant","non_coding_transcript_exon_variant",
                "nc_transcript_variant","non_coding_transcript_variant","NMD_transcript_variant", "incomplete_terminal_codon_variant", "gene_variant","chromosome"]
def term_rank(term):
    return ranked_terms.index(term)

In [28]:
vep_df["effect"] = vep_df.apply(lambda row: min(row["Consequence"].split("&"), key=term_rank), axis=1)

snpeff_df["effect"] = snpeff_df.apply(lambda row: min(row["Annotation"].split('&'), key=term_rank), axis=1)

In [90]:
vc_vep = vep_df.groupby(['effect']).size()
vc_vep.name = "VEP"
vc_snpeff = snpeff_df.groupby(['effect']).size()
vc_snpeff.name = "SnpEff"
vc_df = pd.DataFrame([vc_vep, vc_snpeff])
print("Annotations\n")
print(vc_df.transpose().fillna(0))
impact_vep = vep_df.groupby(['Impact']).size()
impact_vep.name = "VEP"
impact_snpeff = snpeff_df.groupby(['Putative_impact']).size()
impact_snpeff.name = "SnpEff"
impact_df = pd.DataFrame([impact_vep, impact_snpeff])
print("\nImpacts")
print(impact_df.transpose())
counts_vep = vep_df.count()
counts_vep.name = 'VEP'
counts_snpeff = snpeff_df.count()
counts_snpeff.name = 'SnpEff'
counts_df = pd.DataFrame([counts_vep, counts_snpeff])
print("\nCounts")
print(counts_df.transpose())

Annotations

                                                     VEP    SnpEff
3_prime_UTR_variant                               6636.0    6680.0
5_prime_UTR_premature_start_codon_gain_variant       0.0     286.0
5_prime_UTR_variant                               9969.0   10259.0
coding_sequence_variant                             64.0       0.0
conservative_inframe_deletion                        0.0    4619.0
conservative_inframe_insertion                       0.0    8650.0
disruptive_inframe_deletion                          0.0    8990.0
disruptive_inframe_insertion                         0.0   17211.0
downstream_gene_variant                          99177.0   99185.0
exon_loss_variant                                    0.0       7.0
frameshift_variant                              113706.0  113693.0
incomplete_terminal_codon_variant                    8.0       0.0
inframe_deletion                                 13604.0       0.0
inframe_insertion                                

This example is interesting. Vep is usually a pretty general `protein_altering_variant` annotation. Snpeff breaks it down to `disruptive_inframe_insertion`.

In [98]:
vep_df[vep_df['effect'].str.contains('protein_altering_variant')][:1]

,#CHROM,POS,REF,ALT,Consequence,Impact,Symbol,Gene,Feature_type,Feature,Biotype,Hgvsc,Hgvsp,Hgnc_id,effect
19511,7,117120152,C,CCGA,protein_altering_variant,MODERATE,CFTR,ENSG00000001626,Transcript,ENST00000003084,protein_coding,ENST00000003084.6:c.4_5insCGA,ENSP00000003084.6:p.Gln2delinsProLys,1884,protein_altering_variant


In [85]:
vep_df.ix[19511]

,#CHROM,POS,REF,ALT,Consequence,Impact,Symbol,Gene,Feature_type,Feature,Biotype,Hgvsc,Hgvsp,Hgnc_id,effect
19511,7,117120152,C,CCGA,protein_altering_variant,MODERATE,CFTR,ENSG00000001626,Transcript,ENST00000003084,protein_coding,ENST00000003084.6:c.4_5insCGA,ENSP00000003084.6:p.Gln2delinsProLys,1884,protein_altering_variant
19511,7,117120152,C,CCGA,protein_altering_variant,MODERATE,CFTR,ENSG00000001626,Transcript,ENST00000426809,protein_coding,ENST00000426809.1:c.4_5insCGA,ENSP00000389119.1:p.Gln2delinsProLys,1884,protein_altering_variant
19511,7,117120152,C,CCGA,intron_variant,MODIFIER,CFTR,ENSG00000001626,Transcript,ENST00000446805,protein_coding,ENST00000446805.1:c.-191+404_-191+405insCGA,,1884,intron_variant
19511,7,117120152,C,CCGA,protein_altering_variant,MODERATE,CFTR,ENSG00000001626,Transcript,ENST00000454343,protein_coding,ENST00000454343.1:c.4_5insCGA,ENSP00000403677.1:p.Gln2delinsProLys,1884,protein_altering_variant
19511,7,117120152,C,CCGA,intron_variant&non_coding_transcript_variant,MODIFIER,CFTR,ENSG00000001626,Transcript,ENST00000546407,processed_transcript,ENST00000546407.1:n.166+4290_166+4291insCGA,,1884,intron_variant


In [94]:
snpeff_df.ix[19511]

,#CHROM,POS,REF,ALT,Annotation,Putative_impact,Gene_id,Feature_type,Feature_id,Transcript_biotype,Hgvsc,Hgvsp,effect
23333,7,117144330,A,ATCG,disruptive_inframe_insertion,MODERATE,ENSG00000001626,transcript,ENST00000003084,protein_coding,c.77_78insTCG,p.Lys26delinsAsnArg,disruptive_inframe_insertion
23333,7,117144330,A,ATCG,disruptive_inframe_insertion,MODERATE,ENSG00000001626,transcript,ENST00000454343,protein_coding,c.77_78insTCG,p.Lys26delinsAsnArg,disruptive_inframe_insertion
23333,7,117144330,A,ATCG,disruptive_inframe_insertion,MODERATE,ENSG00000001626,transcript,ENST00000426809,protein_coding,c.77_78insTCG,p.Lys26delinsAsnArg,disruptive_inframe_insertion
23333,7,117144330,A,ATCG,5_prime_UTR_variant,MODIFIER,ENSG00000001626,transcript,ENST00000446805,protein_coding,c.-167_-166insTCG,,5_prime_UTR_variant
23333,7,117144330,A,ATCG,non_coding_transcript_exon_variant,MODIFIER,ENSG00000001626,transcript,ENST00000546407,processed_transcript,n.190_191insTCG,,non_coding_transcript_exon_variant


So this example is interesting. It shows two things:
1. SnpEff is providing these "structural_interaction_variant" annotations, which vep does not provide
2. SnpEff is a bit more granular annotating inframe variants

In [81]:
snpeff_df[snpeff_df['effect'].str.contains('structural_interaction_variant')][:1]

,#CHROM,POS,REF,ALT,Annotation,Putative_impact,Gene_id,Feature_type,Feature_id,Transcript_biotype,Hgvsc,Hgvsp,effect
58085,7,117182112,TTAA,T,structural_interaction_variant,HIGH,ENSG00000001626,interaction,2PZG:B_388-B_567:ENST00000003084,protein_coding,c.1160_1162delTAA,,structural_interaction_variant


In [79]:
vep_df.ix[58085]

,#CHROM,POS,REF,ALT,Consequence,Impact,Symbol,Gene,Feature_type,Feature,Biotype,Hgvsc,Hgvsp,Hgnc_id,effect
58085,7,117182112,TTAA,T,inframe_deletion,MODERATE,CFTR,ENSG00000001626,Transcript,ENST00000003084,protein_coding,ENST00000003084.6:c.1160_1162delTAA,ENSP00000003084.6:p.Leu387_Thr388delinsSer,1884,inframe_deletion
58085,7,117182112,TTAA,T,inframe_deletion,MODERATE,CFTR,ENSG00000001626,Transcript,ENST00000426809,protein_coding,ENST00000426809.1:c.1070_1072delTAA,ENSP00000389119.1:p.Leu357_Thr358delinsSer,1884,inframe_deletion
58085,7,117182112,TTAA,T,inframe_deletion,MODERATE,CFTR,ENSG00000001626,Transcript,ENST00000454343,protein_coding,ENST00000454343.1:c.1160_1162delTAA,ENSP00000403677.1:p.Leu387_Thr388delinsSer,1884,inframe_deletion


In [83]:
snpeff_df.ix[58085]

,#CHROM,POS,REF,ALT,Annotation,Putative_impact,Gene_id,Feature_type,Feature_id,Transcript_biotype,Hgvsc,Hgvsp,effect
58085,7,117182112,TTAA,T,structural_interaction_variant,HIGH,ENSG00000001626,interaction,2PZG:B_388-B_567:ENST00000003084,protein_coding,c.1160_1162delTAA,,structural_interaction_variant
58085,7,117182112,TTAA,T,disruptive_inframe_deletion,MODERATE,ENSG00000001626,transcript,ENST00000003084,protein_coding,c.1160_1162delTAA,p.Leu387_Thr388delinsSer,disruptive_inframe_deletion
58085,7,117182112,TTAA,T,disruptive_inframe_deletion,MODERATE,ENSG00000001626,transcript,ENST00000454343,protein_coding,c.1160_1162delTAA,p.Leu387_Thr388delinsSer,disruptive_inframe_deletion
58085,7,117182112,TTAA,T,disruptive_inframe_deletion,MODERATE,ENSG00000001626,transcript,ENST00000426809,protein_coding,c.1070_1072delTAA,p.Leu357_Thr358delinsSer,disruptive_inframe_deletion


In [92]:
snpeff_df = snpeff_df[~snpeff_df['effect'].str.contains('structural_interaction_variant')]

In [100]:
collapse_map = {
'3_prime_UTR_variant': '3_prime_UTR_variant', 
'5_prime_UTR_premature_start_codon_gain_variant': '5_prime_UTR_premature_start_codon_gain_variant',
'5_prime_UTR_variant': '5_prime_UTR_variant',
'coding_sequence_variant': 'coding_sequence_variant',
'conservative_inframe_deletion': 'inframe_deletion',
'conservative_inframe_insertion': 'inframe_insertion',
'disruptive_inframe_deletion': 'inframe_deletion',
'disruptive_inframe_insertion': 'inframe_insertion',
'downstream_gene_variant': 'downstream_gene_variant',
'exon_loss_variant': 'exon_loss_variant',
'frameshift_variant': 'frameshift_variant',
'incomplete_terminal_codon_variant': 'incomplete_terminal_codon_variant',
'inframe_deletion': 'inframe_deletion',
'inframe_insertion': 'inframe_insertion', 
'initiator_codon_variant': 'initiator_codon_variant',
'intergenic_region': 'intergenic_region',
'intron_variant': 'intron_variant',
'missense_variant': 'missense_variant',
'non_coding_transcript_exon_variant': 'non_coding_transcript_exon_variant',
'non_coding_transcript_variant': 'non_coding_transcript_variant',
'protein_altering_variant': 'inframe_insertion',
'sequence_feature': 'sequence_feature',
'splice_acceptor_variant': 'splice_acceptor_variant',
'splice_donor_variant': 'splice_donor_variant',
'splice_region_variant': 'splice_region_variant',
'start_lost': 'start_lost',
'stop_gained': 'stop_gained',
'stop_lost': 'stop_lost', 
'stop_retained_variant': 'stop_retained_variant',
'synonymous_variant': 'synonymous_variant',
'upstream_gene_variant': 'upstream_gene_variant'}

In [103]:
vep_df['effect'] = vep_df['effect'].apply(lambda eff: collapse_map[eff])
snpeff_df['effect'] = snpeff_df['effect'].apply(lambda eff: collapse_map[eff])

In [106]:
vc_vep = vep_df.groupby(['effect']).size()
vc_vep.name = "VEP"
vc_snpeff = snpeff_df.groupby(['effect']).size()
vc_snpeff.name = "SnpEff"
vc_df = pd.DataFrame([vc_vep, vc_snpeff])
print("Annotations\n")
print(vc_df.transpose().fillna(0))

Annotations

                                                     VEP    SnpEff
3_prime_UTR_variant                               6636.0    6680.0
5_prime_UTR_premature_start_codon_gain_variant       0.0     286.0
5_prime_UTR_variant                               9969.0   10259.0
coding_sequence_variant                             64.0       0.0
downstream_gene_variant                          99177.0   99185.0
exon_loss_variant                                    0.0       7.0
frameshift_variant                              113706.0  113693.0
incomplete_terminal_codon_variant                    8.0       0.0
inframe_deletion                                 13604.0   13609.0
inframe_insertion                                25795.0   25861.0
initiator_codon_variant                              0.0       8.0
intergenic_region                                    0.0    1403.0
intron_variant                                  417918.0  416946.0
missense_variant                                 